In [1]:
## imports
import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
import gradio as gr

In [2]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

In [3]:
## Log in to HuggingFace
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
openai = OpenAI()

## Setup the system and user prompts

In [5]:
def make_context(similars, prices):
    message = "To provide some context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [6]:
def messages_for(item, similars, prices):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"
    user_prompt += "How much does this cost?\n\n" + item
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

## Access the Vector Store

In [7]:
DB = "vectorstore"

In [8]:
client = chromadb.PersistentClient(path = DB)
collection = client.get_or_create_collection('products')

In [9]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [10]:
def vector(item):
    return model.encode([item])

In [11]:
def find_similars(item):
    results = collection.query(query_embeddings=vector(item).astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [12]:
documents, prices = find_similars("iphone 16 pro 256 GB")

In [13]:
print(messages_for("Iphone 16 pro 128 GB", documents, prices))

[{'role': 'system', 'content': 'You estimate prices of items. Reply only with the price, no explanation'}, {'role': 'user', 'content': 'To provide some context, here are some other items that might be similar to the item you need to estimate.\n\nPotentially related product:\nUpgraded MOCREO Indoor Wireless Thermometer Hygrometer Sensor, Accessories, Accurate Temperature Humidity Monitor for Freezer Refrigerator Greenhouse Basement Home, MOCREO Hub Required\nMOCREO Indoor Thermometer Hygrometer Sensor (needs to work with MOCREO Hub) with App Alerts, Accurate Temperature Humidity Monitor for Greenhouse Basement Home. Remotely Monitors and Alerts the Temperature and Humidity by Free iPhone/Android Apps, Monitor from Anywhere, Anytime Doesn’t Support 5G WiFi. WIFI Temp Gauge Humidity Sensor Monitor for Home, get exact temperature and humidity. Support storing temperature and humidity data for 2 years in Cloud and data exporting in CSV format for free. Historical data is displayed by a scal

In [14]:
def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return str(match.group()) if match else 0 ## This should be string (if we want to show output in a gradio dashboard)

## GPT 4o Mini Model

In [15]:
def gpt_4o_mini_rag(item):
    documents, prices = find_similars(item)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini", 
        messages = messages_for(item, documents, prices),
        seed = 42,
        max_tokens = 5
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [16]:
gpt_4o_mini_rag("Iphone 16 pro 128 GB")

'999.00'

In [17]:
view = gr.Interface(
    fn = gpt_4o_mini_rag,
    inputs = [gr.Textbox(label="Item Description:")],
    outputs = [gr.Textbox(label = "Price:")],
    flagging_mode = "never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Create an "Agent" class

In [18]:
from agents.frontier_agent import FrontierAgent

In [19]:
import logging
root = logging.getLogger()
root.setLevel(logging.INFO)

In [20]:
agent = FrontierAgent(collection)

INFO:root:[Frontier Agent] Initializing Frontier Agent
INFO:root:[Frontier Agent] Frontier Agent is setting up with OpenAI
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent] Frontier Agent is ready


In [21]:
agent.price("Iphone 16 pro 128 GB")

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-4o-mini with context including 5 similar products
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $999.00


999.0